In [1]:
using SelfConsistentHartreeFock, SecondQuantizedAlgebra
using SelfConsistentHartreeFock.MatrixEquations
using SelfConsistentHartreeFock: @unpack
using LinearAlgebra
import SecondQuantizedAlgebra as SQA
using Symbolics

In [2]:
h = FockSpace(:cavity)

@qnumbers a::Destroy(h)
@variables G::Real Δ::Real K::Real κ::Real

H = -Δ * a' * a + K * (a'^2 * a^2) + G * (a'*a' + a*a)

(-Δ*(a′*a)+K*(a′*a′*a*a)+G*(a′*a′)+G*(a*a))

## Fluctuations
We assume the driven modes $a$ to have a large coherent population $\alpha$ with small quantum fluctuations $d$ on top, so we write $a = \alpha + d$.
$$
\begin{align}
H_{\rm lin}&=(-\alpha  \Delta +2 K |\alpha|^2 \alpha+2G\alpha^*)  d^{\dagger }
+(- \alpha^* \Delta + 2 K |\alpha|^2 \alpha ^* +2G\alpha)d\\
H_{\rm higher order}&=(-\Delta +4 K  |\alpha|^2 ) d^{\dagger } d
+\alpha ^2 K d^{\dagger } d^{\dagger }
+K \left(\alpha ^*\right)^2 d d
+2 K \alpha ^* d^{\dagger } d d
+2 \alpha  K d^{\dagger } d^{\dagger } d
+K d^{\dagger } d^{\dagger } d d
\end{align}
$$

## Hartree-Fock approximation 

We apply the Hartree-Fock approximation to the higher order terms, which consists in replacing products of operators by products of operators and their expectation values:
$$
\begin{align}
d^{\dagger } d^{\dagger } d d
&\approx 4n d^{\dagger } d + m^* d d + m d^{\dagger } d^{\dagger }\\
d^\dagger d d
&\approx 2n d + m d^\dagger\\
\end{align}
$$
such that, up to additive constants (which only shift the zero of energy), the higher order terms become quadratic:
$$
\begin{align}
H_{\rm HF}&=(-\Delta +4 K  |\alpha|^2 +4 K n) d^{\dagger } d
+K (\alpha ^2 +  m) d^{\dagger } d^{\dagger }
+K( \left(\alpha ^*\right)^2 +m^*) d d
\end{align}
$$
Here, we defined $\langle d^{\dagger } d\rangle=n \in \mathbb{R}$ and $\langle d d\rangle = m  \in \mathbb{C}$. The linear terms instead becomes
$$
\begin{align}
H_{\rm lin}&=
(-\alpha  \Delta +2 K |\alpha|^2 \alpha  +2 K m \alpha ^*+4 K n \alpha +2G\alpha^*)  d^{\dagger }
+(-\Delta  \alpha ^*+2 K |\alpha|^2  \alpha ^*+2 \alpha K m^*+4 n K  \alpha ^*+2G\alpha)d
\end{align}
$$

In [3]:
sys = HartreeFockSystem(H, [a], [κ])
collect_dict(sys.H)

Dict{Any, Any} with 6 entries:
  (a′*a′) => G + K*⟨a*a⟩ + K*(⟨a⟩^2)
  (a′*a)  => -Δ + 4K*⟨a′*a⟩ + 4K*⟨a′⟩*⟨a⟩
  (a*a)   => G + K*⟨a′*a′⟩ + K*(⟨a′⟩^2)
  a′      => 2G*⟨a′⟩ - ⟨a⟩*Δ + 2K*⟨a′⟩*⟨a*a⟩ + 4K*⟨a⟩*⟨a′*a⟩ + 2K*⟨a′⟩*(⟨a⟩^2)
  a       => 2G*⟨a⟩ - ⟨a′⟩*Δ + 4K*⟨a′⟩*⟨a′*a⟩ + 2K*⟨a⟩*⟨a′*a′⟩ + 2K*(⟨a′⟩^2)*⟨a⟩
  1.0     => G*(⟨a′⟩^2) + G*(⟨a⟩^2) - ⟨a′⟩*⟨a⟩*Δ + K*(⟨a′⟩^2)*(⟨a⟩^2)

In [4]:
sys.operators |> display
sys.correlators |> display

1-element Vector{Destroy}:
 a

2-element Vector{SecondQuantizedAlgebra.QMul}:
 (a′*a)
 (a*a)

We can rewrite the Hartree-Fock Hamiltonian in matrix (Numba) form:
$$
H_{\rm HF}=\frac{1}{2} \Psi^\dagger 
\begin{pmatrix} A & B \\ B^* & A^T \end{pmatrix}
\Psi 
\quad\text{with}\quad
\Psi =\begin{pmatrix} d \\ d^\dagger \end{pmatrix}
$$
where $A=d^\dagger$ is hermitian and $B=B^T$ is symmetric. In our case, they are given by 
$$
\begin{align}
A&=-\Delta +4 K  |\alpha|^2 +4 K n\\
B&=2G +K (\alpha ^2 +  m)\\
\end{align}
$$

In [5]:
sys.dynamical_matrix.A |> display
sys.dynamical_matrix.B |> display

1×1 Matrix{Num}:
 -Δ + 4K*⟨a′*a⟩ + 4K*⟨a′⟩*⟨a⟩

1×1 Matrix{Num}:
 2(G + K*⟨a*a⟩ + K*(⟨a⟩^2))

Introduce the bosonic symplectic matrix $\Sigma=\mathrm{diag}(I,-I)$.
In Heisenberg–Langevin form with Markov damping rates $\kappa_i$ (pack them in $K\equiv \mathrm{Diag}(\kappa_1,\dots,\kappa_N))$:
$$
\dot \Psi = M\Psi + \Xi
 \qquad
M = -i\Sigma\mathcal{H} - \tfrac{1}{2}
\begin{pmatrix} K & 0\\ 0 & K \end{pmatrix}.
$$
Explicitly,
$$
M=\begin{pmatrix}
-iA-\tfrac12 K & -iB \\
i B^* & i A^T-\tfrac12 K
\end{pmatrix}.
$$
A steady state exists if all eigenvalues of $M$ have negative real parts (stability). For independent thermal baths with occupancies $\bar n_i$,
$$
\Xi = \binom{\sqrt{K},d^{\rm in}}{\sqrt{K},d^{\rm in,\dagger}},\quad
\langle d^{\rm in}(t),d^{\rm in,\dagger}(t')\rangle=(\bar n+1)\delta(t-t'), \quad
\langle d^{\rm in,\dagger}(t),d^{\rm in}(t')\rangle=\bar n\delta(t-t'),
$$
which gives the (white) diffusion matrix
$$
D=\begin{pmatrix}
K(\bar n+1) & 0\\
0 & K\bar n
\end{pmatrix}.
$$
Here $\bar n=\mathrm{Diag}(\bar n_1\dots\bar n_N)$. Zero temperature: set $\bar n=0.$ Define the normally-ordered correlation matrix
$$
C\equiv\langle \Psi,\Psi^\dagger\rangle
=\begin{pmatrix}
\langle d,d^\dagger\rangle & \langle d,d\rangle\\
\langle d^\dagger d^\dagger\rangle & \langle d^\dagger d\rangle
\end{pmatrix}
=\begin{pmatrix}
I + N^\top & M_{an}\\
M_{an}^\dagger & N
\end{pmatrix},
$$
where $N_{ij}=\langle d_j^\dagger d_i\rangle$ and $(M_{an})_{ij}=\langle d_i d_j\rangle$.
Then $C$ satisfies the continuous-time Lyapunov equation
$$
\boxed{\quad M C + CM^\dagger + D = 0 \quad}
$$
Solve this once (no iteration) to get all steady-state second moments. Extract:
$$
N = C_{22}, \qquad M_{an}=C_{12}.
$$

In [ ]:
p = Dict(G => 0.01, Δ => 0.0, K => 0.001, κ => 0.005)
problem = IterativeProblem(sys, p)

IterativeProblem

In [ ]:
α = ComplexF64[1.0+1im, 0.0, 0.0]
@unpack M, D = problem.dynamical_matrix;
_M = M(α)
_D = D(α)
_M |> display
_D

2×2 Matrix{ComplexF64}:
 -0.0025-0.008im    0.004-0.02im
   0.004+0.02im   -0.0025+0.008im

2×2 Matrix{Float64}:
 0.005  0.0
 0.0    0.0

In [10]:
eigvals(_M) |> sum

-0.005000000000000001 - 3.469446951953614e-18im

In [11]:
sys.mean_field_eom

2-element Vector{Num}:
 2G*⟨a⟩ - ⟨a′⟩*Δ + 4K*⟨a′⟩*⟨a′*a⟩ + 2K*⟨a⟩*⟨a′*a′⟩ + 2K*(⟨a′⟩^2)*⟨a⟩
   2G*⟨a′⟩ - ⟨a⟩*Δ + 2K*⟨a′⟩*⟨a*a⟩ + 4K*⟨a⟩*⟨a′*a⟩ + 2K*⟨a′⟩*(⟨a⟩^2)

Your equations are
$$
\begin{aligned}
0&=2G a -\Delta a' + 4K a' m_{a'a} + 2K a m_{a'a'} + 2K a'^2 a\\
0&=2G a' -\Delta a + 2K a' m_{aa} + 4K a m_{a'a} + 2K a a^2.
\end{aligned}
$$
Split linear in $a,a'$ vs nonlinear:
$$
L
\begin{bmatrix} a \ a' \end{bmatrix}
= -N(a,a')
$$
where
$$
L=
\begin{pmatrix}
2G+2K m_{a'a'} & -\Delta+4K m_{a'a}\\
-\Delta+4Km_{a'a} & 2G+2Km_{aa}
\end{pmatrix},
\quad
N(a,a')=
\begin{bmatrix}
2K a'^2 a\\
2K a a^2
\end{bmatrix}.
$$

Define the fixed-point map
$$
T(x)= - L^{-1}N(x) \qquad x=\begin{bmatrix}a \\ a'\end{bmatrix}.
$$
Then iterate
$$
x^{(k+1)} = T \big(x^{(k)}\big)
= - L^{-1} N \big(x^{(k)}\big),
$$

In [19]:
α = ComplexF64[1.0+1im, 0.0, 0.0]
@unpack L, N, Fs = problem.meanfield;
L(α) |> display
N(α) |> display
Fs(α)

2×2 Matrix{ComplexF64}:
 0.02-0.0im   0.0-0.0im
  0.0-0.0im  0.02+0.0im

2-element Vector{ComplexF64}:
 0.004 - 0.004im
 0.004 + 0.004im

2-element Vector{Int64}:
 0
 0

In [13]:
Dampening = 1.0
MaxIter = 1_000
ConvergenceMetricThreshold = 1e-3
alg_kwargs = (;Algorithm = :Anderson, Dampening, MaxIter, ConvergenceMetricThreshold)

α0 = ComplexF64[rand(ComplexF64), 0.0, 0.0]
fixed_point(problem, α0)

Dict{SymbolicUtils.BasicSymbolic{SecondQuantizedAlgebra.AvgSym}, ComplexF64} with 3 entries:
  ⟨a*a⟩  => -0.0534524+0.0641985im
  ⟨a⟩    => 0.0+0.0im
  ⟨a′*a⟩ => -0.513588+0.0im

In [ ]:
Δsweep = range(-0.01, 0.03, 101)


results_up = parameter_sweep(problem, Δ, Δsweep, α0; alg_kwargs...);
results_down = parameter_sweep(problem, Δ, reverse(Δsweep), α0; alg_kwargs...);

In [17]:
results_up[1]

FixedPointAcceleration.FixedPointResults{Float64, Float64}(missing, missing, 0.19115956384755536, :InvalidInputOrOutputOfIteration, 3, [1.8240014437933785, 1.3395561138819294, 0.19115956384755536], FixedPointAcceleration.FunctionEvaluationResult{Float64, Float64}([NaN, NaN, -0.6290726871595487, NaN, 0.22331116686951713, 0.1584904946791036], missing, missing, :InputNAsDetected), [0.9081701091358824 0.0 0.0; 0.9013097922006567 0.0 0.0; … ; 0.0 1.3955801965433434 0.056024082661414044; 0.0 0.6836201138641883 0.1203006892105459], [0.0 0.0 0.0; 0.0 0.0 0.0; … ; 1.3955801965433434 0.056024082661414044 0.2471836465089694; 0.6836201138641883 0.1203006892105459 0.16126783038526607])

In [10]:
amplitude_up = map(results_up) do result
    result[SQA.average(a)] |> norm
end
amplitude_down = map(results_down) do result
    result[SQA.average(a)] |> norm
end

fluctuation_up = map(results_up) do result
    result[SQA.average(a'*a)] |> norm
end
fluctuation_down = map(results_down) do result
    result[SQA.average(a'*a)] |> norm
end
anomalous_up = map(results_up) do result
    result[SQA.average(a*a)] |> norm
end
anomalous_down = map(results_down) do result
    result[SQA.average(a*a)] |> norm
end;

MethodError: MethodError: no method matching getindex(::FixedPointAcceleration.FixedPointResults{Float64, Float64}, ::SymbolicUtils.BasicSymbolic{SecondQuantizedAlgebra.AvgSym})
The function `getindex` exists, but no method is defined for this combination of argument types.

In [11]:
amplitude_up

UndefVarError: UndefVarError: `amplitude_up` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [12]:
using Plots
l = length(amplitude_down)
plt1 = plot(Δsweep, amplitude_up; xlabel = "Detuning Δ", ylabel = "Amplitude", legend = false)
plot!(reverse(Δsweep)[1:l], amplitude_down)

plt2 = plot(Δsweep, fluctuation_up; xlabel = "Detuning Δ", ylabel = "Fluctuation", legend = false)
plot!(reverse(Δsweep)[1:l], fluctuation_down)

plt3 = plot(Δsweep, anomalous_up; xlabel = "Detuning Δ", ylabel = "Anomalous", legend = false)
plot!(reverse(Δsweep)[1:l], anomalous_down)

plot(plt1, plt2, plt3; layout = (3, 1), size=(500, 600))

UndefVarError: UndefVarError: `amplitude_down` not defined in `Main`
Suggestion: check for spelling errors or missing imports.